In [1]:
import matplotlib
matplotlib.use('Agg')

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pickle
import random

from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization

In [2]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
        if K.image_data_format() == 'channels_first':
            inputShape = (depth, height, width)
            chanDim = 1
            
        model.add(Conv2D(32, (3, 3), padding='same', use_bias=False, activation='relu', input_shape=inputShape))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding='same', use_bias=False, activation='relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(classes, activation='softmax'))
        
        return model

In [3]:
EPOCHS = 100
INIT_LR = 5e-4
BS = 32
IMAGE_DIMS = (200, 200, 3)

data = []
labels = []

print('[INFO] loading images...')
imagePaths = sorted(list(paths.list_images('dataset')))
random.seed(42)
random.shuffle(imagePaths)

[INFO] loading images...


In [4]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [5]:
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)
print('[INFO] data matrix: {:.2f}MB'.format(data.nbytes / (1024 * 1000.0)))

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

[INFO] data matrix: 2382.19MB


In [6]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1,
                        shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

In [7]:
print('[INFO] compiling model...')
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0], depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

print('[INFO] training network...')
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS,
    verbose=1
)

[INFO] compiling model...
[INFO] training network...
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 63 steps, validate on 509 samples
Epoch 1/100
63/63 [==============================] - 77s 1s/step - loss: 1.8030 - accuracy: 0.5055 - val_loss: 3.0204 - val_accuracy: 0.1532
Epoch 2/100
63/63 [==============================] - 12s 196ms/step - loss: 1.2401 - accuracy: 0.6215 - val_loss: 2.1745 - val_accuracy: 0.2240
Epoch 3/100
63/63 [==============================] - 12s 194ms/step - loss: 1.1597 - accuracy: 0.6455 - val_loss: 3.5789 - val_accuracy: 0.2554
Epoch 4/100
63/63 [==============================] - 12s 194ms/step - loss: 1.0231 - accuracy: 0.6745 - val_loss: 6.6010 - val_accuracy: 0.2004
Epoch 5/100
63/63 [==============================] - 12s 195ms/step - loss: 1.0306 - accuracy: 0.6840 - val_loss: 2.2411 - val_accuracy: 0.3399
Epoch 6/100
63/63 [==============================] - 12s 195ms/step - loss: 0.9785 - 

In [10]:
model.save('model_20200504_1120.h5')

f = open('label.pickle', 'wb')
f.write(pickle.dumps(lb))
f.close()

In [13]:
plt.style.use('ggplot')
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history['loss'], label='train_loss')
plt.plot(np.arange(0, N), H.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, N), H.history['accuracy'], label='train_accuracy')
plt.plot(np.arange(0, N), H.history['val_accuracy'], label='val_accuracy')
plt.title('Training Loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend(loc='upper left')
plt.savefig('plot_20200504_1120.png')